In [10]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\22498\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
symptoms = ["fever", "cough", "headache", "nausea", "fatigue", "diarrhea"]

# tokenize input text into words
input_text = "I have a fever and cough"
words = word_tokenize(input_text.lower())

# extract symptoms from input text
input_symptoms = []
for word in words:
    if word in symptoms:
        input_symptoms.append(word)

print(input_symptoms)

['fever', 'cough']


In [5]:
import nltk
import numpy as np
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')

def tokenize_words(sentences):
    """
    Tokenize the given list of sentences into words and return a list of the words.
    
    Args:
        sentences (list of str): The sentences to be tokenized.
    
    Returns:
        list of str: The list of words.
    """
    return nltk.word_tokenize(sentences)

def stem(word):
    """
    Reduce a word to its stem using the Porter stemming algorithm.
    
    Args:
        word (str): The word to be stemmed.
    
    Returns:
        str: The stemmed word.
    """
    stemmer = nltk.PorterStemmer()
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    """
    Convert a tokenized sentence to a bag of words vector.
    
    Args:
        tokenized_sentence (list of str): The tokenized sentence to be converted to a bag of words vector.
        words (list of str): The list of all words in the dataset.
    
    Returns:
        bag (ndarry): The bag of words vector.
    """
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, word in enumerate(words):
        if word in sentence_words:
            bag[idx] = 1.0
    return bag

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\22498\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [135]:
import tensorflow as tf
import pickle
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.models import Sequential
import numpy as np
import random
import json
from nltk.tokenize import word_tokenize

def load_data():
    with open('intents.json') as file:
        data = json.load(file)
    return data

def create_tokenize():
    data = load_data()

    words = []
    labels = []
    docs_x = []
    docs_y = []
    ignore_words = ["?", ".", "!"]

    # loop through each sentence in the intents data
    for intent in data['intents']:
        for pattern in intent['patterns']:
            # tokenize each word in the sentence
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            # add the tokenized words to docs_x list
            docs_x.append(wrds)
            # add the corresponding tag to docs_y list
            docs_y.append(intent['tag'])

        if intent['tag'] not in labels:
            # add the tag to the labels list
            labels.append(intent['tag'])

    # remove duplicates and sort the words list
    # words = sorted(list(set(words)))
    # sort the labels list
    labels = sorted(list(set(labels)))

    # # perform stemming and lower each word and remove duplicates
    words = [stem(w.lower()) for w in words if w not in ignore_words]
    words = list(set(words))

    # # remove duplicates
    # classes = list(set(classes))

    return words, labels, docs_x, docs_y

def create_training_data():
    words, labels, docs_x, docs_y = create_tokenize()
    # create a bag of words model
    training = []
    output = []
    out_empty = [0] * len(labels)

    for x, doc in enumerate(docs_x):
        bag = []
        # tokenize each word in the sentence
        wrds = [stem(w.lower()) for w in doc]
        # create a bag of words model for the sentence
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        # create the output row for the sentence
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    # convert the training data to numpy arrays
    training = np.array(training)
    output = np.array(output)

    return training, output

# define the neural network model
def train_model():
    training, output = create_training_data()
    print("THIS IS THE X:", training)
    model = Sequential()
    model.add(Dense(128, input_shape=(len(training[0]),), activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(len(output[0]), activation='softmax'))
    print("THIS IS THE NUMBER OF OUTPUT: ", len(output[0]))
    # compile the model
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    training, output = create_training_data()
    model.fit(training, output, epochs=200, batch_size=8, verbose=1)

    with open("model.pkl", "wb") as file:
        pickle.dump(model, file)

# train the model
# def train_model():
#     create_model()
#     with open("model.pkl", "rb") as file:
#         model = pickle.load(file)
#     training, output = create_training_data()
#     model.fit(training, output, epochs=200, batch_size=8, verbose=1)
    # model.save('chatbot_model.h5')

def predict_intent(sentence, words, classes):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stem(word.lower()) for word in sentence_words]
    # create bag of words array
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    # predict with the trained model
    with open("model.pkl", "rb") as file:
        model = pickle.load(file)
    results = model.predict(np.array([bag]))[0]
    # get the index of the max value
    index = np.argmax(results)
    # map the index to the class and return
    return classes[index]

def get_response(input_text, model, intents, words, classes, threshold=0.1):
    # preprocess the input text
    sentence_words = word_tokenize(input_text)
    sentence_words = [stem(word.lower()) for word in sentence_words]
    print("1.Tokenize: ", sentence_words)
    # convert input sentence into bag of words
    input_bag = bag_of_words(sentence_words, words)
    # bag = [0] * len(words)
    # for s in sentence_words:
    #     for i, w in enumerate(words):
    #         if w == s:
    #             bag[i] = 1
    print("2.bag of words: ", input_bag)
    # predict the intent for the input using the trained model
    input_bag = np.array(input_bag).reshape(1, -1)
    results = model.predict([input_bag])[0]
    print("This is the input bag: ", input_bag)
    
    # filter out the predictions below the confidence threshold
    results = [[i, r] for i, r in enumerate(results) if r > threshold]
    # sort the results by probability in descending order
    results.sort(key=lambda x: x[1], reverse=True)
    print("This is the result: ", results)
    # initialize an empty response message
    response = "I'm sorry, I didn't understand that. Could you please try again?"
    # check if any prediction meets the confidence threshold

    if results:
        # loop through the filtered results
        print("Results are true.")
        for result in results:
            # get the corresponding intent tag
            tag = classes[result[0]]
            print("This is the tag: ", tag)
            # check if the tag is in the intents dictionary
            for intent in intents['intents']:
                if tag == intent["tag"]:
                    return random.choice(intent['responses'])
            
            # if any(intent['tag'] == tag for intent in intents['intents']):
            #     random.choice(intents['responses'])
            #     # # get the list of possible responses for the tag
            #     # responses = intents['intents'][tag]['responses']
            #     # # select a random response from the list
            #     # response = random.choice(responses)
            #     print("Response picked")
    return response

In [136]:
train_model()

THIS IS THE X: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
THIS IS THE NUMBER OF OUTPUT:  6
Epoch 1/200
6/6 [==============================] - 0s 1ms/step - loss: 1.7817 - accuracy: 0.2708
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 1.6858 - accuracy: 0.4583
Epoch 3/200
6/6 [==============================] - 0s 998us/step - loss: 1.5795 - accuracy: 0.4375
Epoch 4/200
6/6 [==============================] - 0s 1ms/step - loss: 1.4774 - accuracy: 0.4375
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 1.3984 - accuracy: 0.4375
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 1.3279 - accuracy: 0.4167
Epoch 7/200
6/6 [==============================] - 0s 1ms/step - loss: 1.2590 - accuracy: 0.4167
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 1.1938 - accuracy: 0.4583
Epoch 9/200
6/6 [==============================] - 0s 

In [141]:
words, classes, docs_x, docs_y = create_tokenize()
with open("model.pkl", "rb") as file:
    model = pickle.load(file)
with open('intents.json') as file:
    intents = json.load(file)
get_response("Who are you?", model, intents, words, classes, threshold=0.1)

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-05 12:24:22         2050
metadata.json                                  2023-03-05 12:24:22           64
variables.h5                                   2023-03-05 12:24:22        85160
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...vars
1.Tokenize:  ['who', 'are', 'you', '?']
2.bag of words:  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1/1 [==============================] - 0s 34ms/step
This 

'My name is DocBot, a medical chatbot developed by Xufeng Bai. I am able to make predictions on diseases given the symptoms.'

In [20]:
# from nltk.stem import WordNetLemmatizer

# def load_intents(filename):
#     """
#     Load the intents file from disk.

#     Args:
#     filename: string containing the filename of the intents file.

#     Returns:
#     intents: dictionary containing the loaded intents.
#     """
#     with open(filename) as file:
#         intents = json.load(file)
#     return intents

# def create_tokenize(intents):
#     """
#     Create a tokenizer for the intents.

#     Args:
#     intents: dictionary containing the intents.

#     Returns:
#     words: list of unique words in the intents.
#     classes: list of unique classes in the intents.
#     tokenizer: instance of a Tokenizer object fit on the words in the intents.
#     """
#     words = []
#     classes = []
#     documents = []
#     ignore_words = ["?", ".", "!"]
#     lemmatizer = WordNetLemmatizer()

#     for intent in intents["intents"]:
#         for pattern in intent["patterns"]:
#             # Tokenize each word in the pattern and add it to words list
#             w = nltk.word_tokenize(pattern)
#             words.extend(w)
#             # Add the document with its corresponding class
#             documents.append((w, intent["tag"]))
#             # Add the tag to the classes list
#             if intent["tag"] not in classes:
#                 classes.append(intent["tag"])

#         # Lemmatize and lowercase each word and add it to the words list
#         for word in intent["patterns"]:
#             word = lemmatizer.lemmatize(word.lower())
#             if word not in ignore_words:
#                 words.append(word)

#     # Remove duplicates and sort the words and classes lists
#     words = sorted(list(set(words)))
#     classes = sorted(list(set(classes)))

#     # Create a tokenizer for the words
#     tokenizer = nltk.tokenize.Tokenizer()
#     tokenizer.fit_on_texts(words)

#     return words, classes, tokenizer

# def create_model(words, tags, tokenized_patterns):
#     """
#     Create the neural network model for the chatbot.
    
#     Args:
#     words (list): List of all the unique tokenized words.
#     tags (list): List of all the unique tags.
#     tokenized_patterns (list): List of tuples where each tuple contains a tokenized pattern and its corresponding tag.
    
#     Returns:
#     model (Sequential): The neural network model for the chatbot.
#     """
#     # Create training data
#     training_data = []
#     output_data = []
#     # Create an array of zeros for each tag
#     output_empty = [0] * len(tags)
#     for tokenized_pattern in tokenized_patterns:
#         # Create bag of words for each pattern
#         bag_of_words = []
#         pattern_words = tokenized_pattern[0]
#         for word in words:
#             bag_of_words.append(1) if word in pattern_words else bag_of_words.append(0)
#         # Create output row for the corresponding tag
#         output_row = list(output_empty)
#         output_row[tags.index(tokenized_pattern[1])] = 1
#         # Append the bag of words and output row to the training data
#         training_data.append(bag_of_words)
#         output_data.append(output_row)
#     # Convert the training data and output data to numpy arrays
#     X_train = np.array(training_data)
#     y_train = np.array(output_data)
#     # Create the neural network model
#     model = Sequential()
#     model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))

#     return model
    
# def train_model(intents_file, model_file):
#     """Train a neural network model using the intents file and save it to a file.

#     Args:
#         intents_file (str): Path to the intents JSON file.
#         model_file (str): Path to the file where the trained model will be saved.

#     Returns:
#         None
#     """
#     # Load the intents file
#     intents = load_intents(intents_file)

#     # Create the tokenizer and the training data
#     tokenizer, X_train, y_train = create_tokenize(intents)

#     # Create and train the model
#     model = create_model(X_train, y_train)
#     model.fit(X_train, y_train, epochs=200, batch_size=5, verbose=1)
#     model.save(model_file)

# def predict_intent(sentence, words, classes, model):
#     # tokenize the pattern
#     sentence_words = nltk.word_tokenize(sentence)
#     # stem each word
#     sentence_words = [stem(word.lower()) for word in sentence_words]
#     # create bag of words array
#     bag = [0] * len(words)
#     for s in sentence_words:
#         for i, w in enumerate(words):
#             if w == s:
#                 bag[i] = 1
#     # predict with the trained model
#     results = model.predict(np.array([bag]))[0]
#     # get the index of the max value
#     index = np.argmax(results)
#     # map the index to the class and return
#     return classes[index]


# def get_response(intents_json, sentence, model, words, classes):
#     # predict the intent of the sentence
#     intent = predict_intent(sentence, words, classes, model)
#     # get a random response from the intent's responses
#     for i in intents_json['intents']:
#         if i['tag'] == intent:
#             return random.choice(i['responses'])

In [96]:
create_model()

THIS IS THE X: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 1 0]
 [0 1 0 ... 0 1 0]
 [0 1 0 ... 0 0 1]]
THIS IS THE NUMBER OF OUTPUT:  6
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-04 16:53:02         2019
metadata.json                                  2023-03-04 16:53:02           64
variables.h5                                   2023-03-04 16:53:02        83200


c:\Users\22498\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
